In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
device = torch.device("cpu")

In [3]:
import pandas as pd

# 거래 내역 엑셀파일 읽기
df = pd.read_excel("./transaction.xlsx")

# 거래일이 없는 행 제거 (null 또는 빈 문자열)
df = df[df["거래일"].notna() & (df["거래일"].str.strip() != "")]

# JSON 파일로 저장
df.to_json("output.json", orient="records", force_ascii=False, indent=2)

In [4]:
from dotenv import load_dotenv
import openai
from openai import OpenAI
import os

load_dotenv()

os.environ["WANDB_DISABLE_CODE"] = "true"
api_key = os.getenv("OPENAI_API_KEY")
client=OpenAI(api_key=api_key)

In [5]:
import wandb
import logging

wandb.init(project="CardList")
wandb.run.name = "sft"

wandb: Currently logged in as: hongppa324 (hongppa324-bizbee) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
from datasets import load_dataset

# JSON 데이터 로드
dataset = load_dataset("json", data_files="output.json")["train"]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
print(dataset)

Dataset({
    features: ['거래일', '카드구분', '이용카드', '가맹점명', '금액', '이용구분', '거래통화', '해외이용금액', '취소상태'],
    num_rows: 748
})


In [8]:
store_names = dataset["가맹점명"]
unique_stores = list(set(store_names))
unique_stores

['테이큰커피',
 '서초백화점약국',
 '라이즈커피로스터즈',
 '에이스손해보험 다이렉트',
 '현대푸드(또봉이통닭&별난만두)',
 '지금연구소 나우카페',
 'BITE 2 HEAVEN(바이투헤븐)',
 '빽다방 건대동문회관점',
 '시옌',
 '서울붕어베이커리',
 '포크포크',
 '우청옥',
 '건대365약국',
 '수리치킨',
 '이케아코리아 유한회사',
 '백패커',
 '롯데슈퍼 서초국제전자센터점',
 '파덕 오리바베큐',
 '부리또삐아',
 '영커피 강남해커스점',
 '페이플',
 '원족발 순대국',
 '한설원',
 '홍등',
 '고흥순대국',
 '(주)케이에프씨코리아 KFC강남역',
 '(주)아트박스 건대스타시티점',
 '대보건설(주) 구리(일산방향)주',
 '서일체육문화센터',
 '제오헤어(신림점)',
 '현이네 고기국수',
 '다와 마트',
 '세븐스타 코인노래연습장',
 '왓더버거 서초점',
 '24시자양감자탕',
 '홈플러스금천점',
 '(주)써브웨이 신림본점',
 '주식회사 아성다이소',
 '노다메',
 '서울옥',
 '주식회사 함흥면옥',
 '움버거앤윙스 어린이대공원역점',
 '쇼군',
 '엑스엑스커피메이커 건대점',
 '세븐일레븐 M신대방역점',
 '메가엠지씨커피 신대방역점',
 '주식회사 놀유니버스',
 '지에스GS25 서초한신점',
 '주식회사 비케이알 버거킹신림역',
 '( 주)버거요강남본점',
 '이디야 신대방역점',
 '주식회사 편한커피',
 '구시아푸드마켓',
 '티머니 개인택시',
 '맘스터치 대학동점',
 '서울베스트의료의원',
 '제니스약국',
 '한송',
 '린스시',
 '주식회사 한빛',
 '쿠팡이츠',
 '(주)갈라인터내셔널',
 '이마트24 센트럴키오스크점',
 '고벤트 주식회사',
 '길동우동 건대점',
 '나이스인프라 주식회사',
 '노브랜드버거 신림남부점',
 '런드리킹7호점',
 '우먼센스',
 '서초제일마디의원',
 '청담',
 '함포해장 서초점',
 '주식회사 서울밥집',
 '빈브라더스 합정점',
 '

In [10]:
label_list = ["음식", "카페", "쇼핑", "디지털", "교통", "의료", "보험", "통신", "공과금", "생활용품", "오락", "기타"]
# label2id = {label: i for i, label in enumerate(label_list)}
# id2label = {i: label for label, i in label2id.items()}

In [11]:
def classify_store(store):
    prompt = f"""
    가맹점명: {store}
    이 가맹점의 업종을 다음 중 하나로 분류하세요: [음식, 카페, 쇼핑, 디지털, 교통, 의료, 보험, 통신, 공과금, 생활용품, 오락, 기타]

    결과는 업종명만 한 단어로 출력하세요.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error: {e}")
        return "기타"

In [13]:
results = []
for store in unique_stores:
    category = classify_store(store).strip()
    # label_id = label2id.get(category, label2id["기타"])
    print(f"{store} → {category}")
    results.append({
        "store": store, 
        "category": category,
        # "label_id": label_id
    })

테이큰커피 → 카페
서초백화점약국 → 의료
라이즈커피로스터즈 → 카페
에이스손해보험 다이렉트 → 보험
현대푸드(또봉이통닭&별난만두) → 음식
지금연구소 나우카페 → 카페
BITE 2 HEAVEN(바이투헤븐) → 음식
빽다방 건대동문회관점 → 카페
시옌 → 음식
서울붕어베이커리 → 음식
포크포크 → 음식
우청옥 → 음식
건대365약국 → 의료
수리치킨 → 음식
이케아코리아 유한회사 → 쇼핑
백패커 → 쇼핑
롯데슈퍼 서초국제전자센터점 → 쇼핑
파덕 오리바베큐 → 음식
부리또삐아 → 음식
영커피 강남해커스점 → 카페
페이플 → 디지털
원족발 순대국 → 음식
한설원 → 음식
홍등 → 음식
고흥순대국 → 음식
(주)케이에프씨코리아 KFC강남역 → 음식
(주)아트박스 건대스타시티점 → 쇼핑
대보건설(주) 구리(일산방향)주 → 기타
서일체육문화센터 → 오락
제오헤어(신림점) → 기타
현이네 고기국수 → 음식
다와 마트 → 쇼핑
세븐스타 코인노래연습장 → 오락
왓더버거 서초점 → 음식
24시자양감자탕 → 음식
홈플러스금천점 → 쇼핑
(주)써브웨이 신림본점 → 음식
주식회사 아성다이소 → 쇼핑
노다메 → 음식
서울옥 → 음식
주식회사 함흥면옥 → 음식
움버거앤윙스 어린이대공원역점 → 음식
쇼군 → 음식
엑스엑스커피메이커 건대점 → 카페
세븐일레븐 M신대방역점 → 쇼핑
메가엠지씨커피 신대방역점 → 카페
주식회사 놀유니버스 → 오락
지에스GS25 서초한신점 → 쇼핑
주식회사 비케이알 버거킹신림역 → 음식
( 주)버거요강남본점 → 음식
이디야 신대방역점 → 카페
주식회사 편한커피 → 카페
구시아푸드마켓 → 쇼핑
티머니 개인택시 → 교통
맘스터치 대학동점 → 음식
서울베스트의료의원 → 의료
제니스약국 → 의료
한송 → 음식
린스시 → 음식
주식회사 한빛 → 기타
쿠팡이츠 → 음식
(주)갈라인터내셔널 → 기타
이마트24 센트럴키오스크점 → 쇼핑
고벤트 주식회사 → 기타
길동우동 건대점 → 음식
나이스인프라 주식회사 → 기타
노브랜드버거 신림남부점 → 음식
런드리킹7호점 → 기타
우먼센

In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

embeddings = OpenAIEmbeddings()

# Document 생성
docs = [
    Document(page_content=store["store"], metadata={"category": store["category"]})
    for store in results
]

# 벡터 DB 생성
vectorstore = FAISS.from_documents(docs, embedding=embeddings)

# 저장
vectorstore.save_local("./vector_db/stores")

/var/folders/yz/dp_hy1vd5jj8pzrlts8w9z4m0000gn/T/ipykernel_34885/3666435399.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [15]:
# 저장된 DB 확인
os.listdir("./vector_db/stores")

['index.faiss', 'index.pkl']

In [16]:
# 벡터 DB 로드
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

# embedding model
embedding_model = OpenAIEmbeddings()

# vector store 로드
vectorstore = FAISS.load_local(
    "./vector_db/stores",
    embedding_model,
    allow_dangerous_deserialization=True
)

# 유사한 문맥 가져오기
def get_similar_context(store_name: str, k=3, max_chars=500):
    docs = vectorstore.similarity_search(store_name, k=k)
    return "\n".join([f"- {doc.page_content}" for doc in docs])[:max_chars]

# 추론
def rag_infer(store_name: str, context: str):
    prompt = f"""[QUESTION]
    "{store_name}"는 어떤 업종인가요?

    [CONTEXT]
    {context}

    [ANSWER]
    인터넷 검색을 활용하세요.
    업종은 다음 중 하나로만 답하세요: [음식, 카페, 쇼핑, 디지털, 교통, 의료, 보험, 통신, 공과금, 생활용품, 오락, 기타]
    결과는 업종명만 한 단어로 출력하세요.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        generated = response.choices[0].message.content.strip()
    except Exception as e:
        print(f"OpenAI API 오류: {e}")
        return "기타"

    # 정제
    generated = generated.split("\n")[0].strip()
    label_set = {"음식", "카페", "쇼핑", "디지털", "교통", "의료", "보험", "통신", "공과금", "생활용품", "오락", "기타"}

    if generated not in label_set:
        print(f"'{generated}' 는 유효하지 않은 결과 → 기타 유지")
        return "기타"

    return generated

# 결과 갱신
for entry in results:
    if entry["category"] == "기타":
        print(f"\n기타 재분류 시도: {entry['store']}")
        context = get_similar_context(entry["store"], k=3)
        if not context.strip():
            print("context 없음 → 기타 유지")
            continue
        new_category = rag_infer(entry["store"], context)
        print(f" {entry['store']} → {new_category}")
        entry["category"] = new_category

# 최신 results를 기반으로 문서 재생성
updated_docs = [
    Document(page_content=entry["store"], metadata={"category": entry["category"]})
    for entry in results
]

# 벡터스토어 재생성
updated_vectorstore = FAISS.from_documents(updated_docs, embedding=embedding_model)

# 기존 경로에 덮어쓰기 저장
updated_vectorstore.save_local("./vector_db/stores")


기타 재분류 시도: 대보건설(주) 구리(일산방향)주
 대보건설(주) 구리(일산방향)주 → 기타

기타 재분류 시도: 제오헤어(신림점)
 제오헤어(신림점) → 기타

기타 재분류 시도: 주식회사 한빛
 주식회사 한빛 → 기타

기타 재분류 시도: (주)갈라인터내셔널
 (주)갈라인터내셔널 → 기타

기타 재분류 시도: 고벤트 주식회사
 고벤트 주식회사 → 기타

기타 재분류 시도: 나이스인프라 주식회사
 나이스인프라 주식회사 → 디지털

기타 재분류 시도: 런드리킹7호점
 런드리킹7호점 → 생활용품

기타 재분류 시도: 시카프관광개발주식회사함양(통
 시카프관광개발주식회사함양(통 → 기타

기타 재분류 시도: (주)팀플러스
 (주)팀플러스 → 기타

기타 재분류 시도: (주)어비즈
 (주)어비즈 → 기타

기타 재분류 시도: 서울시설공단(인터넷정기권)
 서울시설공단(인터넷정기권) → 공과금

기타 재분류 시도: OF
 OF → 기타

기타 재분류 시도: (주)공유어장
 (주)공유어장 → 기타

기타 재분류 시도: 아칸&플렉스 대전로드점
 아칸&플렉스 대전로드점 → 오락

기타 재분류 시도: (주)핌아시아
 (주)핌아시아 → 기타

기타 재분류 시도: 런드리킹셀프빨래방 신대방점
 런드리킹셀프빨래방 신대방점 → 생활용품

기타 재분류 시도: 서울도시가스(주)
 서울도시가스(주) → 공과금

기타 재분류 시도: 크린토피아 관악신사점
 크린토피아 관악신사점 → 생활용품

기타 재분류 시도: 천지인주유소
 천지인주유소 → 교통

기타 재분류 시도: (주)코리아세븐 신림고시촌점
 (주)코리아세븐 신림고시촌점 → 쇼핑

기타 재분류 시도: 라콜롬브코리아 주식회사
 라콜롬브코리아 주식회사 → 카페

기타 재분류 시도: 와이즐리
 와이즐리 → 생활용품

기타 재분류 시도: 씨유 관악건영점
 씨유 관악건영점 → 생활용품

기타 재분류 시도: 에프이지
 에프이지 → 기타

기타 재분류 시도: 관악구청
 관악구청 → 기타

기타 재분류 시도: 조이(JOY)
 조이(JOY) → 오락



In [17]:
import json

corpus = [
    {
        "instruction": "다음 가맹점의 업종을 분류하세요.",
        "input": entry["store"],
        "output": entry["category"]
    }
    for entry in results
]

with open("./corpus.json", "w", encoding="utf-8") as f:
    json.dump(corpus, f, ensure_ascii=False, indent=2)

In [18]:
corpus

[{'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '테이큰커피', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서초백화점약국', 'output': '의료'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '라이즈커피로스터즈', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '에이스손해보험 다이렉트',
  'output': '보험'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '현대푸드(또봉이통닭&별난만두)',
  'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '지금연구소 나우카페', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': 'BITE 2 HEAVEN(바이투헤븐)',
  'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '빽다방 건대동문회관점', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '시옌', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서울붕어베이커리', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '포크포크', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '우청옥', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '건대36

In [19]:
import random

# shuffle한 뒤 분할
random.shuffle(corpus)
split_idx = int(len(corpus) * 0.8)
train_data = corpus[:split_idx]
validation_data = corpus[split_idx:]

In [20]:
train_data

[{'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '하오차이', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '컴포즈커피 테헤란아이파크점',
  'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '나카노라멘', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '더벤티 신림역점', 'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '사운즈커피 프리미어 강남',
  'output': '카페'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '서초칼국수 닭한마리', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '감초식당', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': 'BITE 2 HEAVEN(바이투헤븐)',
  'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '청담', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '조이(JOY)', 'output': '오락'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '소문난 청년푸줏간', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': 'CU 서초한빛점', 'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '카페동네 신림

In [21]:
validation_data

[{'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '나이스인프라 주식회사',
  'output': '디지털'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '고기반장', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '지에스25(GS25) 서초모나코점',
  'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '씨유(CU) 관악신사점',
  'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '포크포크', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '채향', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '세븐일레븐 대전가양행복점',
  'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '백패커', 'output': '쇼핑'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '주식회사 에스씨케이컴퍼니',
  'output': '기타'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': '세븐스타 코인노래연습장',
  'output': '오락'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.', 'input': '순대실록남부터미널점', 'output': '음식'},
 {'instruction': '다음 가맹점의 업종을 분류하세요.',
  'input': 'CU 건대상허도서관점',
  'output': '생활용품'},
 {'instruction': '다음 가맹점의 업종

In [22]:
from datasets import Dataset, DatasetDict

# 데이터셋 변환
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(validation_data)

raw_datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset
})

In [23]:
# 데이터 전처리
def preprocess_function(example):
    return {
        "text": f"[INSTRUCTION] {example['instruction']}\n[INPUT] {example['input']}\n[OUTPUT] {example['output']}"
    }

raw_datasets = raw_datasets.map(preprocess_function)

Map:   0%|          | 0/211 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [24]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B")
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B")

In [25]:
# 데이터 포맷 변환 함수
def format_llama_prompt(example):
    prompt = f"### 질문: {example['instruction']}\n### 답변: {example['output']}"
    return {"text": prompt}

# 토크나이징 함수
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=80
        # max_length=128
    )
    tokenized["labels"] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in input_ids]
        for input_ids in tokenized["input_ids"]
    ]
    return tokenized

In [26]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    if isinstance(predictions, tuple):
        predictions = predictions[0]

    preds = predictions.argmax(axis=-1) if predictions.ndim == 3 else predictions
    preds = preds.tolist()
    labels = labels.tolist()

    # -100 제거
    labels = [[token for token in label if token != -100] for label in labels]

    # 디코딩
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # 부분 일치 평가 (예: 쇼핑 vs 쇼핑입니다)
    correct = sum(
        any(label in pred for label in label_list) and true in pred
        for pred, true in zip(decoded_preds, decoded_labels)
    )
    total = len(decoded_labels)

    print("decoded_preds", decoded_preds)
    print("decoded_labels", decoded_labels)

    return {
        "accuracy": correct / total if total > 0 else 0.0
    }

In [27]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

Map:   0%|          | 0/211 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

In [28]:
import wandb

wandb.init(project="CardList")
wandb.run.name = "sft"

In [29]:
from transformers import TrainingArguments, Trainer, default_data_collator

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="llama-finetune-1b",
    gradient_checkpointing=True,
    use_cpu=True
)

In [31]:
device

device(type='cpu')

In [32]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

/var/folders/yz/dp_hy1vd5jj8pzrlts8w9z4m0000gn/T/ipykernel_34885/1686238893.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.611900,1.278851,0.000000
2,0.913900,1.365949,0.000000
3,0.523500,1.449974,0.000000


decoded_preds ['[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울�\n\n대라멘식회사\n[OUTPUT] 음지털\n���������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울대\n대\n[OUTPUT] 음식회사���������������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울세이\n\n)\n)\n)\n약초점점\n리아\n[OUTPUT] 쇼핑점�����������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울\n\n)\n)\n세악\n대점\n[OUTPUT] 음핑점대��������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울마\n라\n[OUTPUT] 음식회사���������������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울이\n[OUTPUT] 음식회사�����������������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울븐\n\n\n�\n대\n점점국\n[OUTPUT] 음핑점대�����������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을 분류하세요.\n[INPUT] 서울대\n\n[OUTPUT] 음핑점����������������������������������������������������', '[INSTRUCTION] 다음 가맹점의 업종을

TrainOutput(global_step=633, training_loss=0.9983645975118941, metrics={'train_runtime': 11174.053, 'train_samples_per_second': 0.057, 'train_steps_per_second': 0.057, 'total_flos': 295680714670080.0, 'train_loss': 0.9983645975118941, 'epoch': 3.0})

In [33]:
trainer.save_model("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

('./finetuned_model/tokenizer_config.json',
 './finetuned_model/special_tokens_map.json',
 './finetuned_model/tokenizer.json')

In [34]:
model = AutoModelForCausalLM.from_pretrained("./finetuned_model").to(device)
tokenizer = AutoTokenizer.from_pretrained("./finetuned_model")
tokenizer.pad_token = tokenizer.eos_token

# validation 데이터셋 평가 및 저장
import csv
import json

def evaluate_and_save(dataset, output_path="validation_predictions.csv"):
    model.eval()
    results = []

    print(f"\n=== 전체 validation 예측 시작 (총 {len(dataset)}건) ===")
    for i, example in enumerate(dataset):
        prompt = f"[INSTRUCTION] {example['instruction']}\n[INPUT] {example['input']}\n[OUTPUT]"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_new_tokens=10,
                # max_new_tokens=50,
                do_sample=False,
                temperature=0.0,
            )

        generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        predicted = generated.split("[OUTPUT]")[-1].strip() if "[OUTPUT]" in generated else generated.strip()

        result = {
            "index": i + 1,
            "store": example["input"],
            "label": example["output"],
            "predict": predicted,
            "correct": predicted == example["output"]
        }

        print(f"[{result['index']}] {result['store']} → {result['predict']} ({'✅' if result['correct'] else '❌'})")
        results.append(result)

    # CSV 저장
    with open(output_path, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=results[0].keys())
        writer.writeheader()
        writer.writerows(results)

    # JSON 저장 (옵션)
    with open(output_path.replace(".csv", ".json"), "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    print(f"\n 결과 저장 완료: {output_path}, {output_path.replace('.csv', '.json')}")

evaluate_and_save(raw_datasets["validation"], output_path="validation_predictions.csv")

/opt/homebrew/Caskroom/miniforge/base/envs/hhplus_week7/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/hhplus_week7/lib/python3.13/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



=== 전체 validation 예측 시작 (총 53건) ===


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[1] 나이스인프라 주식회사 → 음식당 (❌)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

[50] 계경순대국(신대방점) → 음식당 (❌)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[51] 멘쇼 → 음식당 (❌)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


[52] (주)케이에프씨코리아 KFC강남역 → 쇼핑핑 (❌)
[53] 크린토피아 관악신사점 → 카페 (❌)

 결과 저장 완료: validation_predictions.csv, validation_predictions.json
